In [129]:
from whisperweranalysis.structure import LJSpeech
from whisperweranalysis.groundtruth import lj_valid
from whisperweranalysis.wer import compute

In [130]:
from whisperweranalysis.mel_utilities import audio2mel

In [131]:
# audio2mel()

In [132]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Natural Speech

In [133]:
GT = LJSpeech.load_from_dir('../whisperweranalysis/LJ_Valid_transcription/GT/0')

In [134]:
compute(lj_valid, GT.transcriptions)

2.5423728813559325

## Vocoded Speech

In [135]:
GT_Mel = LJSpeech.load_from_dir('../whisperweranalysis/LJ_Valid_transcription/GT/GT_MEL')

In [136]:
compute(lj_valid, GT_Mel.transcriptions)

1.8765133171912836

In [137]:
with open('temp_NS.txt', 'w') as f:
    f.writelines([f'{x}\n' for x in GT.transcriptions])
    
with open('temp_NS_Vocoded.txt', 'w') as f:
    f.writelines([f'{x}\n' for x in GT_Mel.transcriptions])
    